In [1]:
import pandas as pd

from valuation.asset.identity.dataset import DatasetID
from valuation.core.stage import DatasetStage
from valuation.infra.store.dataset import DatasetStore
store = DatasetStore()
dataset_id = DatasetID(name="train_val", stage=DatasetStage.MODEL)
passport = store.get_passport(dataset_id=dataset_id)
ds = store.get(passport=passport)
train_df = ds.data
# Check frequency of the 'ds' column in your train_df
# Calculate the difference between consecutive dates for a sample series
sample_id = train_df['unique_id'].iloc[0]
sample_series_dates = train_df[train_df['unique_id'] == sample_id]['ds'].sort_values()
date_diffs = sample_series_dates.diff().dropna()

print("Frequency check for a sample series:")
print(date_diffs.value_counts())

# Check for missing periods per series
completeness_check = train_df.groupby('unique_id')['ds'].agg(['min', 'max', 'count'])
expected_weeks_train = (train_df['ds'].max() - train_df['ds'].min()).days // 7 + 1 # Approximate for 6 years

print(f"\nExpected number of weeks in train_df (approx): {expected_weeks_train}")
print("\nChecking completeness per series:")
print(completeness_check.head())

# Find series that don't have the expected number of weeks
incomplete_series = completeness_check[completeness_check['count'] != expected_weeks_train]
if not incomplete_series.empty:
    print(f"\nFound {len(incomplete_series)} series with potentially missing periods:")
    print(incomplete_series.head())
else:
    print("\nAll series appear to have the expected number of periods.")

2025-10-23 21:33:42.174 | DEBUG    | valuation.asset.dataset.base:load:338 - Dataset Dataset train_val of the model stage created on 2025-10-23 at 21:11 loaded.


Frequency check for a sample series:
ds
7 days     304
35 days      1
21 days      1
Name: count, dtype: int64

Expected number of weeks in train_df (approx): 313

Checking completeness per series:
                            min        max  count
unique_id                                        
100_analgesics       1990-01-03 1995-12-27    307
100_bath soap        1992-02-26 1995-12-27    195
100_bathroom tissues 1990-01-03 1995-12-27    303
100_beer             1991-06-12 1995-12-27    232
100_bottled juices   1990-01-03 1995-12-27    307

Found 1924 series with potentially missing periods:
                            min        max  count
unique_id                                        
100_analgesics       1990-01-03 1995-12-27    307
100_bath soap        1992-02-26 1995-12-27    195
100_bathroom tissues 1990-01-03 1995-12-27    303
100_beer             1991-06-12 1995-12-27    232
100_bottled juices   1990-01-03 1995-12-27    307


## History per Series

In [2]:
# How much history do you have per series?
series_length = train_df.groupby('unique_id')['ds'].agg(['min', 'max', 'count'])
series_length['weeks'] = (series_length['max'] - series_length['min']).dt.days / 7

print("Series length distribution:")
print(series_length['weeks'].describe())
print(f"\nSeries with < 260 weeks: {(series_length['weeks'] < 260).sum():,}")
print(f"Series with >= 260 weeks: {(series_length['weeks'] >= 260).sum():,}")

Series length distribution:
count    2403.000000
mean      280.491469
std        52.452566
min        30.000000
25%       237.000000
50%       312.000000
75%       312.000000
max       312.000000
Name: weeks, dtype: float64

Series with < 260 weeks: 664
Series with >= 260 weeks: 1,739
